## k-means Clustering
クラスタの数を$k$として、以下のアルゴリズムでクラスタリングを行う。

1. 各データ点$x_1, \ldots, x_n$に対してランダムにクラスタを割り振る。
2. 割り振ったデータ点の中心（多くは算術平均）$V_1, \ldots, V_k$を求める。
3. $x_i$と$V_j$を比較して$x_i$を最も近い中心のクラスタに割り当てる
4. 上記を指定したiterationか割り当てが収束したと判定されるまで繰り返す。

（参考: [wikipedia](https://ja.wikipedia.org/?curid=74044))

In [ ]:
# prepare data
n = 100000
d = 64
x = np.random.random((n, d)).astype('float32')
x[:, 0] += np.arange(n) / 1000.

### kmeans
L2ノルムを近さの基準とした配列。([他の基準](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes))

```python
import faiss
clus = faiss.Clustering(d, k)
```

params: 
- `d`: `int`, ベクトルの次元(dimension of each vector)
- `k`: `int`, クラスタの数(=中心点の個数)(number of clusters)

instance:
- `.verbose`: `bool=False`, ログを表示するか(whether display log)
- `.niter`: `int=25`, イテレーション(iteration)
- `.max_points_per_centroid`: `int=256`, 各クラスタに登録されるデータ点の最大数(max points per centroid)
- `.centroids`: `faiss.swigfaiss_gpu.FloatVector`, 中心点のデータ(resulting centroids), `shape=(k, d)`
- `.obj`: `faiss.swigfaiss_gpu.FloatVector`,  イテレーションとその時の目的関数の値(values of the objective function along iterations), `shape=(niter)`
- `.train(x, index_ceontroids)`:  kmeansを実行する(append vectors to the index)
  - params:
     - `x`: `np.ndarray`, `n`個のベクトルを格納した配列(database of vectors), `shape=(n, d)`
     - `index_centroids`: `faiss.GpuIndexFlatL2`, 中心点のデータに対するインデックス配列

In [ ]:
# prepare gpu index
res = faiss.StandardGpuResources()
flat_config = faiss.GpuIndexFlatConfig()
index_centroids = faiss.GpuIndexFlatL2(res, d, flat_config)

In [ ]:
# run k-means clustering
k = 1000
clus = faiss.Clustering(d, k)
clus.verbose = False
clus.niter = 20
clus.max_points_per_centroid = 100000
clus.train(x, index_centroids)

#### 便利関数

```
faiss.vector_float_to_array(vector)
```

params:
- `vector`: `faiss.swigfaiss_gpu.FloatVector`, ベクトル, `shape=(n*d)`

return:
- `array`: `np.ndarray`, 1次元numpy配列, `shape=(n*d)`

普通は出力を`array.reshape(n, d)`として、形を整える。

In [ ]:
# faiss vector to np.ndarray
print(faiss.vector_float_to_array(clus.centroids).reshape(k, d))
print(faiss.vector_float_to_array(clus.obj))

[[79.60448     0.5593583   0.4300954  ...  0.5860559   0.50424546
   0.44380125]
 [42.66248     0.6328918   0.4065965  ...  0.40675142  0.42998046
   0.57096714]
 [36.276512    0.51245147  0.56459254 ...  0.6085555   0.4264428
   0.42930868]
 ...
 [ 3.5409474   0.54657286  0.49638352 ...  0.5877457   0.37224233
   0.4481677 ]
 [31.704718    0.5278905   0.41830444 ...  0.4697383   0.443506
   0.29364893]
 [ 7.4414115   0.3552784   0.46254358 ...  0.6049498   0.62158626
   0.3990567 ]]
[806450.6  506579.47 500436.5  498116.9  496898.84 496151.28 495626.9
 495243.25 494955.47 494740.2  494569.3  494442.3  494325.16 494236.
 494164.16 494097.1  494051.66 494013.53 493977.6  493946.9 ]


#### 結果を見る
データ点`x`がどの中心点のクラスタに属するかを求めるには中心点のデータ`clus.centroids`に対応するインデックス配列`index_centroids`を使用して`.search(x, 1)`を行うことで求められる。

また、中心点に最も近いデータを求める場合は、逆に`x`に対応するインデックス配列`index`を用意して、`index.search(faiss.vector_to_array(clus.centroids).reshape(k, d), 1)`とすれば良い。

In [ ]:
D, I = index_centroids.search(x, 1)

In [ ]:
print(I[:5])
print(D[:5])

In [ ]:
# data point nearest to each centroids
index = faiss.GpuIndexFlatL2(res, d, flat_config)
index.add(x)
D_nearest, I_nearest = index.search(faiss.vector_to_array(clus.centroids).reshape(k, d), 1)

In [ ]:
print(I_nearest[:5])
print(D_nearest[:5])

[[309]
 [313]
 [397]
 [ 51]
 [753]]
[[978]
 [363]
 [841]
 [987]
 [978]]
